In [ ]:
import pandas as pd

train = pd.read_csv('orders_seafood_new.csv')

# Выбираем числовые признаки
num_features = ["reject_count", "confirm_count", "last_summ", "summ_", "count_position"]

for col in num_features:
    print(f"\nПризнак: {col}")
    desc = train[col].describe()
    
    mean = desc["mean"]
    std = desc["std"]
    
    # Считаем выбросы как значения, которые выходят за пределы 3 стандартных отклонений
    outliers = train[(train[col] < mean - 3*std) | (train[col] > mean + 3*std)]
    print(f"Количество выбросов (>|3σ|): {len(outliers)}")

In [ ]:
results = pd.DataFrame(grid.cv_results_)
print(results[['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_test_f1']].sort_values('mean_test_f1', ascending = False).head(10))

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve, CalibrationDisplay
import matplotlib.pyplot as plt

# 1. Генерация данных
X, y = make_classification(n_samples=500, n_features=10, n_informative=5, random_state=42)

# 2. Разделение данных на train и val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Обучение модели
model = LogisticRegression(max_iter=1000, random_state=42)
# Дополните код обучением модели
model.fit(X_train, y_train)

# 4. Получение вероятностей положительного класса
y_pred = model.predict_proba(X_val)[:, 1] # Ваш код здесь

# 5. Калибровочная кривая
prob_true, prob_pred = calibration_curve(y_val, y_pred, n_bins=10) # Ваш код здесь, используйте n_bins=10

# 6. Построение диаграммы
# Ваш код здесь
disp = CalibrationDisplay(
    prob_true, 
    prob_pred, 
    y_val
)
disp.plot()
plt.xlabel("Средняя предсказанная вероятность")
plt.ylabel("Фактическая доля положительных исходов")
plt.title("Диаграмма калибровки")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

def calculate_ece(y_true, y_prob, n_bins=10):
    """
    Вычисляет Expected Calibration Error (ECE) для предсказаний модели.
    
    ECE измеряет, насколько хорошо предсказанные вероятности модели
    соответствуют реальной частоте правильных предсказаний.
    
    Args:
        y_true (np.array): Истинные метки классов (0 или 1).
        y_prob (np.array): Предсказанные вероятности для класса 1.
        n_bins (int): Количество интервалов (бинов) для разбиения вероятностей.
        
    Returns:
        float: Значение ECE.
    """
    # Создаем границы бинов, равномерно распределенные от 0 до 1.
    # Например, для n_bins=10, это будут [0.0, 0.1, 0.2, ..., 1.0].
    bins = np.linspace(0, 1, n_bins + 1)
    
    ece = 0  # Инициализируем общую ошибку калибровки.
    n = len(y_true)  # Общее количество примеров.
    
    print("Расчет ECE по бинам:")
    
    # Итерируемся по каждому бину.
    for i, (bin_lower, bin_upper) in enumerate(zip(bins[:-1], bins[1:])):
        # Создаем логическую маску для отбора примеров, которые попадают в текущий бин.
        # Последний бин (i == n_bins - 1) включает правую границу, чтобы захватить
        # вероятность 1.0, остальные - нет.
        if i == n_bins - 1:
            mask = (y_prob >= bin_lower) & (y_prob <= bin_upper)
        else:
            mask = (y_prob >= bin_lower) & (y_prob < bin_upper)
        
        # Проверяем, есть ли в текущем бине хотя бы один пример.
        if np.sum(mask) > 0:
            bin_size = np.sum(mask)
            
            # Вычисляем среднюю предсказанную уверенность (confidence) для бина.
            bin_conf = np.mean(y_prob[mask])
            
            # Вычисляем среднюю точность (долю правильных ответов) в этом бине.
            bin_acc = np.mean(y_true[mask])
            
            # Вычисляем вклад текущего бина в общую ошибку.
            # Это абсолютная разница, умноженная на количество примеров в бине.
            contribution = np.abs(bin_conf - bin_acc) * bin_size
            ece += contribution
            
            # Выводим подробную информацию о расчетах для каждого бина.
            print(f"Бин {i+1} [{bin_lower:.2f}-{bin_upper:.2f}): размер={bin_size}, уверенность={bin_conf:.3f}, точность={bin_acc:.3f}, вклад={contribution / n:.3f}")
    
    # Возвращаем итоговое значение ECE, нормированное на общее количество примеров.
    return ece / n

# ===============================================
# Пример использования функции
# ===============================================

# Истинные метки (0 или 1)
y_true = np.array([0, 1, 1, 0, 1, 1, 0, 0, 1, 0])

# Вероятности, предсказанные моделью для класса "1"
y_prob = np.array([0.15, 0.25, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 0.35, 0.90])

# Вызываем функцию для расчета ECE с 5 бинами.
ece_value = calculate_ece(y_true, y_prob, n_bins=5)

# Выводим итоговое значение ECE.
print(f"\nИтоговый ECE: {ece_value:.3f}")

In [ ]:
# ---------------------------
# 2. Реализуйте функции калибрации (ECE, MCE)
# ---------------------------
def calculate_ece(y_true, y_prob, n_bins=10):
    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0
    n = len(y_true)
    for i, (bin_lower, bin_upper) in enumerate(zip(bins[:-1], bins[1:])):
        if i == n_bins - 1:
            mask = (y_prob >= bin_lower) & (y_prob <= bin_upper)
        else:
            mask = (y_prob >= bin_lower) & (y_prob < bin_upper)
        if np.sum(mask) > 0:
            bin_conf = np.mean(y_prob[mask])
            bin_acc = np.mean(y_true[mask])
            ece += np.abs(bin_conf - bin_acc) * np.sum(mask)
    return ece / n

def calculate_mce(y_true, y_prob, n_bins=10):
    bins = np.linspace(0, 1, n_bins + 1)
    max_error = 0
    for i, (bin_lower, bin_upper) in enumerate(zip(bins[:-1], bins[1:])):
        if i == n_bins - 1:
            mask = (y_prob >= bin_lower) & (y_prob <= bin_upper)
        else:
            mask = (y_prob >= bin_lower) & (y_prob < bin_upper)
        if np.sum(mask) > 0:
            bin_conf = np.mean(y_prob[mask])
            bin_acc = np.mean(y_true[mask])
            max_error = max(max_error, np.abs(bin_conf - bin_acc)) 
    return max_error

In [ ]:
from time import time
from sklearn.model_selection import cross_validate


# ---------------------------
# 1. Запуск Кросс-валидации
# ---------------------------
results_list = []
print("--- Запуск кросс-валидации ---")

for name, pipe in pipelines.items():
    start_time = time()
    print(f"Оценка модели: {name:<20}...", end=' ')

    # Кросс-валидация на всех данных (X, y)
    cv_res = cross_validate(
        pipe, X, y, # <-- Используем исходные X и y
        scoring=scoring,
        cv=outer_cv,
        n_jobs=-1,
        error_score='raise'
    )

    # Усреднённые метрики
    new_row = {
        'model': name,
        **{f'{m}_mean': cv_res[f'test_{m}'].mean() for m in scoring},
        **{f'{m}_std':  cv_res[f'test_{m}'].std()  for m in scoring},
    }

    results_list.append(new_row)
    end_time = time()
    print(f"Готово за {end_time - start_time:.2f} сек.")

results_df = pd.DataFrame(results_list)

# ---------------------------
# 2. Вывод результатов
# ---------------------------
display_cols = ['model', 'roc_auc_mean', 'accuracy_mean', 'f1_mean', 'precision_mean', 'recall_mean']
results_df = results_df[[col for col in display_cols if col in results_df.columns]].sort_values(by='roc_auc_mean', ascending=False)


def format_row(row):
    roc_auc = row.get('roc_auc_mean', float('nan'))
    acc = row.get('accuracy_mean', float('nan'))
    f1 = row.get('f1_mean', float('nan'))
    precision = row.get('precision_mean', float('nan'))
    recall = row.get('recall_mean', float('nan'))

    return (f"{row['model']:<20} | "
            f"AUC: {roc_auc:.3f} | "
            f"Acc: {acc:.3f} | "
            f"F1: {f1:.3f} | "
            f"Prec: {precision:.3f} | "
            f"Rec: {recall:.3f}")

print("\n" + "="*80)
print("Сравнительные результаты кросс-валидации")
print("="*80)
print("\n".join(results_df.apply(format_row, axis=1)))

In [ ]:
# Получаем веса из обученной модели
weights = logreg_l1.coef_[0]

# Создаём DataFrame для удобного анализа
feature_weights = pd.DataFrame({'feature': X_train_processed.columns, 'weight': weights})

# Фильтруем и сортируем признаки по весам
non_zero_features = feature_weights[feature_weights['weight'] != 0].sort_values(by='weight', ascending=False)
zero_features = feature_weights[feature_weights['weight'] == 0]

print("Отобранные признаки (с ненулевыми весами):")
print(non_zero_features)

print("\nОтброшенные признаки (с нулевыми весами):")
print(zero_features)